## 03: SVC Kerner Linear con Pycaret

In [1]:
!pip install scikit-optimize
!pip install optuna
!pip install pycaret
from pycaret.classification import *
import pandas as pd                                             #Importar libreria pandas para leer archivo parquet
pd.set_option('display.float_format', '{:.2f}'.format)          #Formatro decimal para el display de los datos
from scipy.stats import skew, kurtosis, chi2_contingency          #Importar libreria scipy para calcular skew y kurtosis
import seaborn as sns                                           #Importar libreria seaborn para graficar
from sklearn.model_selection import train_test_split, RandomizedSearchCV            #Importar libreria sklearn para separar datos en train y test
from sklearn.dummy import DummyClassifier                       #Importar libreria sklearn para crear clasificador dummy
from sklearn.metrics import roc_auc_score, roc_curve, auc       #Importar libreria sklearn para calcular metricas de clasificacion binaria
import matplotlib.pyplot as plt                                 #Importar libreria matplotlib para graficar
import numpy as np                                              #Importar libreria numpy para calculos matematicos
from sklearn.pipeline import Pipeline                           #Importar libreria sklearn para crear pipelines
from sklearn.base import BaseEstimator, TransformerMixin        #La clase ColumnSelector hereda de BaseEstimator y TransformerMixin, lo que permite integrarse fácilmente en un pipeline de scikit-learn
from sklearn.impute import SimpleImputer                        #Importar libreria sklearn para imputar valores faltantes
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, MinMaxScaler, RobustScaler #Importar libreria sklearn para escalar y codificar variables
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression             #Importar libreria sklearn para crear modelo de regresion logistica
from sklearn.svm import SVC                                     #Importar libreria sklearn para crear modelo de SVM
from skopt import BayesSearchCV                                 #Importar libreria skopt para optimizar hiperparametros
from skopt.space import Real, Integer                           #Importar libreria skopt para definir espacio de busqueda de hiperparametros
import optuna                                                   #Importar libreria optuna para optimizar hiperparametros
from sklearn.ensemble import RandomForestClassifier             #Importar libreria sklearn para crear modelo de Random Forest
from sklearn.model_selection import train_test_split            #
from sklearn.ensemble import RandomForestClassifier             #
from sklearn.neural_network import MLPClassifier                #Importar libreria sklearn para crear modelo de redes neuronales

Dado los múltiplques inconvenientes que se obtuvieron con la optimización del SVC, al construirlo manualmente, probaremos con Pycaret

In [2]:
df_train= pd.read_parquet ("df_train.parquet")
print(df_train.shape)
train_dataset, test_data_set = train_test_split(df_train, test_size=0.3, random_state=42)
print(train_dataset.shape)
print(test_data_set.shape)


(23494, 46)
(16445, 46)
(7049, 46)


In [3]:
X_test, y_test = test_data_set.drop(columns="Target"), test_data_set["Target"]
print(X_test.shape)
print(y_test.shape)

(7049, 45)
(7049,)


In [4]:
clf = setup(data=train_dataset, target="Target", session_id=42)

,Description,Value
0,Session id,42
1,Target,Target
2,Target type,Binary
3,Original data shape,"(16445, 46)"
4,Transformed data shape,"(16445, 96)"
5,Transformed train set shape,"(11511, 96)"
6,Transformed test set shape,"(4934, 96)"
7,Numeric features,24
8,Categorical features,21
9,Rows with missing values,100.0%


In [5]:
model = create_model('svm')


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5512,0.5037,0.4299,0.2994,0.3529,0.0260,0.0270
1,0.4900,0.5260,0.5945,0.3005,0.3992,0.0331,0.0390
2,0.6794,0.5292,0.1128,0.3217,0.1670,0.0224,0.0271
3,0.4040,0.5214,0.7652,0.2919,0.4226,0.0170,0.0262
4,0.6073,0.4901,0.2317,0.2754,0.2517,-0.0119,-0.0120
5,0.5430,0.5095,0.4024,0.2857,0.3342,0.0013,0.0014
6,0.4318,0.4859,0.6738,0.2878,0.4033,0.0065,0.0088
7,0.7124,0.4735,0.0000,0.0000,0.0000,-0.0052,-0.0323
8,0.6968,0.5409,0.0884,0.3671,0.1425,0.0358,0.0494


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
tuned_model = tune_model(model, optimize='AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6684,0.5186,0.1433,0.3176,0.1975,0.0248,0.0279
1,0.7020,0.5166,0.0701,0.3770,0.1183,0.0317,0.0483
2,0.7116,0.5076,0.0030,0.1667,0.0060,-0.0043,-0.0190
3,0.6560,0.5054,0.1311,0.2792,0.1784,-0.0045,-0.0050
4,0.7011,0.5153,0.0244,0.2500,0.0444,-0.0065,-0.0131
5,0.6725,0.5054,0.1037,0.2906,0.1528,0.0035,0.0042
6,0.7194,0.4950,0.0274,0.6923,0.0528,0.0317,0.0964
7,0.7107,0.4953,0.0671,0.4490,0.1167,0.0460,0.0766
8,0.6759,0.5415,0.1738,0.3585,0.2341,0.0590,0.0652


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [7]:
evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [8]:
print(tuned_model)

SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.2, fit_intercept=True,
              l1_ratio=0.9100000001, learning_rate='adaptive', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1,
              penalty='elasticnet', power_t=0.5, random_state=42, shuffle=True,
              tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)


In [9]:
predictions = predict_model(tuned_model , data=X_test, raw_score=True)
y_pred = predictions['prediction_label']
roc_auc = roc_auc_score(y_test, y_pred)
print(roc_auc)

0.5086873164051889


Finalmente observamos que el modelo con los mejores hiperparámetros de Pycaret no arroja buenos resultados